## Imports

In [ ]:
import numpy as np
import platform
import tempfile
import os
import matplotlib.pyplot as plt
import skimage

## Useful functions

In [ ]:
def viewimage(im, normalise=True, MINI=0.0, MAXI=255.0):
    """
    Display the image in grayscale in GIMP. If GIMP is already open, it will be used.
    By default, the image is normalized between 0 and 255 before being saved.
    If normalise=False, MINI and MAXI will be set to 0 and 255 in the resulting image.

    Args:
        im (ndarray): The input image.
        normalise (boolean): Whether to normalize the image (default: True).
        MINI (float): The minimum value for normalization (default: 0.0).
        MAXI (float): The maximum value for normalization (default: 255.0).

    """

    imt = np.float32(im.copy())
    if platform.system() == "Darwin":  # Mac OS
        prephrase = "open -a GIMP "
        endphrase = " "
    elif platform.system() == "Linux":
        prephrase = "gimp "
        endphrase = " &"
    elif platform.system() == "Windows":
        prephrase = 'start /B "D:/GIMP/bin/gimp-2.10.exe" -a '  # Replace D:/... by your GIMP's path
        endphrase = ""
    else:
        print("Systeme non pris en charge par l affichage GIMP")
        return "erreur d afficahge"
    if normalise:
        m = imt.min()
        imt = imt - m
        M = imt.max()
        if M > 0:
            imt = imt / M

    else:
        imt = (imt - MINI) / (MAXI - MINI)
        imt[imt < 0] = 0
        imt[imt > 1] = 1

    nomfichier = tempfile.mktemp("TPIMA.png")
    commande = prephrase + nomfichier + endphrase
    skimage.io.imsave(nomfichier, imt)
    os.system(commande)


def viewimage_color(im, normalise=True, MINI=0.0, MAXI=255.0):
    """
    This function displays the image in grayscale in GIMP.

    Args:
        im (ndarray): The image to display in grayscale.
        normalise (boolean): Indicates whether the image should be normalized between 0 and 255 before being saved. By default, normalise=True.
        MINI (float): The minimum value of the normalization interval. By default, MINI=0.
        MAXI (float): The maximum value of the normalization interval. By default, MAXI=255.
    """
    imt = np.float32(im.copy())
    if platform.system() == "Darwin":  # on est sous mac
        prephrase = "open -a GIMP "
        endphrase = " "
    elif (
        platform.system() == "Linux"
    ):  # SINON ON SUPPOSE LINUX (si vous avez un windows je ne sais comment faire. Si vous savez dites-moi.)
        prephrase = "gimp "
        endphrase = " &"
    elif platform.system() == "Windows":
        prephrase = 'start /B "D:/GIMP/bin/gimp-2.10.exe" -a '  # Remplacer D:/... par le chemin de votre GIMP
        endphrase = ""
    else:
        print("Systeme non pris en charge par l affichage GIMP")
        return "erreur d afficahge"

    if normalise:
        m = imt.min()
        imt = imt - m
        M = imt.max()
        if M > 0:
            imt = imt / M
    else:
        imt = (imt - MINI) / (MAXI - MINI)
        imt[imt < 0] = 0
        imt[imt > 1] = 1

    nomfichier = tempfile.mktemp("TPIMA.pgm")
    commande = prephrase + nomfichier + endphrase
    skimage.io.imsave(nomfichier, imt)
    os.system(commande)


def strel(forme, taille, angle=45):
    """
    Returns a structuring element of the specified shape, size and eventual orientation.

    Args:
        forme (str): The shape of the structuring element. Possible values are 'diamond', 'disk', 'square', and 'line'.
        taille (int): The size of the structuring element.
        angle (int, optional): The orientation angle of the line structuring element in degrees. Default is 45.

    Returns:
        numpy.ndarray: The generated structuring element.

    Raises:
        RuntimeError: If the specified shape is not recognized.

    Note:
    - The 'diamond' shape represents a closed diamond with a radius of size.
    - The 'disk' shape represents a closed disk with a radius of size.
    - The 'square' shape represents a square with a side length of size.
    - The 'line' shape represents a line segment with a length of size and an orientation angle of angle degrees.
      The angle should be between 0 and 180.
    """

    def _extracted_from_strel_30(angle, taille):
        angle = int(-np.round(angle))
        angle %= 180
        angle = np.float32(angle) / 180.0 * np.pi
        x = int(np.round(np.cos(angle) * taille))
        y = int(np.round(np.sin(angle) * taille))
        if x**2 + y**2 == 0:
            if abs(np.cos(angle)) > abs(np.sin(angle)):
                x = int(np.sign(np.cos(angle)))
                y = 0
            else:
                y = int(np.sign(np.sin(angle)))
                x = 0
        rr, cc = skimage.draw.line(0, 0, y, x)
        rr = rr - rr.min()
        cc = cc - cc.min()
        img = np.zeros((rr.max() + 1, cc.max() + 1))
        img[rr, cc] = 1

        return img

    if forme == "diamond":
        return skimage.morphologylogy.diamond(taille)
    if forme == "disk":
        return skimage.morphology.disk(taille)
    if forme == "square":
        return skimage.morphology.square(taille)
    if forme == "line":
        return _extracted_from_strel_30(angle, taille)

    raise RuntimeError("Erreur dans fonction strel: forme incomprise")


def couleurs_alea(im):
    """
    Generates random colors for a grayscale image.

    Arg:
        im (ndarray): The grayscale image.

    Returns:
        ndarray: The image with random colors.
    """

    sh = im.shape
    out = np.zeros((sh[0], sh[1], 3), dtype=np.uint8)
    nbcoul = np.int32(im.max())
    tabcoul = np.random.randint(0, 256, size=(nbcoul + 1, 3))
    tabcoul[0, :] = 0

    for k in range(sh[0]):
        for l in range(sh[1]):
            out[k, l, :] = tabcoul[im[k, l]]

    return out


def gris_depuis_couleur(im):
    """
    Converts an image (ndarray) with RGB channels into an image (ndarray) with a single grayscale channel.
    """

    return im[:, :, :3].sum(axis=2) / 3

## Examples of functions for this work

In [ ]:
# Binary images
# im=skimage.io.imread('cellbin.bmp')
# im=skimage.io.imread('cafe.bmp')

# Gray-scale images
# im = skimage.io.imread('Images/retina2.gif')
# im=skimage.io.imread('Images/retina2.gif')
im = gris_depuis_couleur(skimage.io.imread("Images/bat200.bmp"))
# im=skimage.io.imread('Images/bulles.bmp')
# im=gris_depuis_couleur (skimage.io.imread('Images/cailloux.png'))
# im=gris_depuis_couleur(skimage.io.imread('Images/cailloux2.png'))
# im=skimage.io.imread('Images/laiton.bmp')

fig, axs = plt.subplots(1, 5, figsize=(25, 25))

# Original image
axs[0].imshow(im, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original Image")

# Structuring element
se = strel("line", 10, -45)

# dilation
dil = skimage.morphology.dilation(im, se)
axs[1].imshow(dil, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Dilation")

# erosion
ero = skimage.morphology.erosion(im, se)
axs[2].imshow(ero, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Erosion")

# opening
opening = skimage.morphology.opening(im, se)
axs[3].imshow(opening, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("Opening")

# closing
close = skimage.morphology.closing(im, se)
axs[4].imshow(close, cmap="gray", vmin=0, vmax=255)
axs[4].set_title("Closing")

## Successive dilations

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15, 15))

# Structuring element
se1 = strel("square", 3)
se2 = strel("square", 5)
se3 = strel("square", 7)

# Successive dilations
dilation_1 = skimage.morphology.dilation(im, se1)
dilation_2 = skimage.morphology.dilation(dilation_1, se2)

# One dilation
dilation_3 = skimage.morphology.dilation(im, se3)

axs[0].imshow(im, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original Image")

axs[1].imshow(dilation_2, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Iterative Dilation")

axs[2].imshow(dilation_3, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Dilation with 8 by 8")

axs[3].imshow(dilation_3 - dilation_2, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("Difference")

## Successive openings

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(15, 15))

# Successive openings
opening_1 = skimage.morphology.opening(im, se1)
opening_2 = skimage.morphology.opening(opening_1, se2)

# One dilation
opening_3 = skimage.morphology.opening(im, se2)

axs[0].imshow(im, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original Image")

axs[1].imshow(opening_2, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Iterative Opening")

axs[2].imshow(opening_3, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Opening with 5 by 5")

axs[3].imshow(opening_3 - opening_2, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("Difference")

## Top-hat transform

In [ ]:
im = gris_depuis_couleur(skimage.io.imread("Images/laiton.bmp"))

fig, axs = plt.subplots(1, 3, figsize=(15, 15))

t = 10
se = strel("disk", 6)

ch = im - skimage.morphology.opening(im, se)

axs[0].imshow(im, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original Image")

axs[1].imshow(skimage.morphology.opening(im, se), cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Opening")

axs[2].imshow(ch, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Top-hat")

## Dual

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 15))

t = 10
se = strel("disk", 6)
ch = skimage.morphology.closing(im, se) - im

axs[0].imshow(im, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original Image")

axs[1].imshow(skimage.morphology.closing(im, se), cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Closing")

axs[2].imshow(ch, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Dual Top-hat")

## Pointwise maximum

In [ ]:
im = skimage.io.imread("Images/retina2.gif")

se1 = strel("line", 10, 0)
se2 = strel("line", 10, 45)
se3 = strel("line", 10, 90)
se4 = strel("line", 10, -45)

opening1 = skimage.morphology.opening(im, se1)
opening2 = skimage.morphology.opening(im, se2)
opening3 = skimage.morphology.opening(im, se3)
opening4 = skimage.morphology.opening(im, se4)

point_max = np.zeros(im.shape)

for row in range(im.shape[0]):
    for col in range(im.shape[1]):
        point_max[row, col] = max(
            opening1[row, col],
            opening2[row, col],
            opening3[row, col],
            opening4[row, col],
        )


fig, axs = plt.subplots(1, 4, figsize=(15, 5))
axs[0].imshow(opening1, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Angle: 0, length = 10")
axs[1].imshow(opening2, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Angle: 45, length = 10")
axs[2].imshow(opening3, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Angle: 90, length = 10")
axs[3].imshow(opening4, cmap="gray", vmin=0, vmax=255)
axs[3].set_title("Angle: -45, length = 10")

In [ ]:
plt.imshow(point_max, cmap="gray", vmin=0, vmax=255)
plt.title("Point-wise maximum")

## Alternate sequential filters

In [ ]:
N = 11
r = 2

fig, axs = plt.subplots(2, 5, figsize=(15, 5))

im = skimage.io.imread("Images/retina2.gif")

for i in range(0, 2):
    for j in range(0, 5):
        axs[i, j].imshow(im, cmap="gray", vmin=0, vmax=255)
        se = strel("square", r)
        opening = skimage.morphology.opening(im, se)
        recon1 = skimage.morphology.reconstruction(opening, im)
        closing = skimage.morphology.closing(recon1, se)
        recon2 = skimage.morphology.reconstruction(closing, recon1, "erosion")
        im = recon2
        r += 1

In [ ]:
plt.imshow(im, cmap="gray", vmin=0, vmax=255)

## Reconstruction

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

im = skimage.io.imread("Images/retina2.gif")
se4 = strel("disk", 4)
open4 = skimage.morphology.opening(im, se4)
reco = skimage.morphology.reconstruction(open4, im)

axs[0].imshow(im, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original Image")
axs[1].imshow(open4, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Opening")
axs[2].imshow(reco, cmap="gray", vmin=0, vmax=255)
axs[2].set_title("Reconstruction")

## Segmentation

In [ ]:
im = gris_depuis_couleur(skimage.io.imread("Images/bat200.bmp"))
se = strel("disk", 1)
dil1 = skimage.morphology.dilation(im, se)
er1 = skimage.morphology.erosion(im, se)
morpho_grad = dil1 - er1
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].imshow(im, cmap="gray", vmin=0, vmax=255)
axs[0].set_title("Original Image")
axs[1].imshow(dil1 - er1, cmap="gray", vmin=0, vmax=255)
axs[1].set_title("Morphological Gradient")

In [ ]:
im = gris_depuis_couleur(skimage.io.imread("Images/bat200.bmp"))
se = skimage.morphology.disk(1)

grad = skimage.morphology.dilation(im, se) - skimage.morphology.erosion(im, se)
grad = np.int32(grad > 40) * grad
plt.imshow(grad, cmap="gray")
plt.show()

local_mini = skimage.feature.peak_local_max(255 - grad)
markers = np.zeros_like(grad, dtype=int)
for i, (row, col) in enumerate(local_mini, start=1):
    markers[row, col] = i

plt.imshow(local_mini, cmap="gray")
plt.show()

labels = skimage.segmentation.watershed(grad, markers, watershed_line=True)
plt.imshow(couleurs_alea(labels))
plt.show()

# visualization of the result
segm = labels.copy()
for i in range(segm.shape[0]):
    for j in range(segm.shape[1]):
        if segm[i, j] == 0:
            segm[i, j] = 255
        else:
            segm[i, j] = 0

# superimposition of segmentation contours on the original image
contourSup = np.maximum(segm, im)
plt.imshow(contourSup, cmap="gray")

In [ ]:
im = gris_depuis_couleur(skimage.io.imread("Images/bat200.bmp"))
se = skimage.morphology.disk(1)
r = 1

for r, _ in enumerate(range(0, 6), start=1):
    se = strel("square", r)
    opening = skimage.morphology.opening(im, se)
    # recon1 = morpho.reconstruction(opening, im)
    closing = skimage.morphology.closing(recon1, se)
    # recon2 = morpho.reconstruction(closing, recon1, 'erosion')
    fas = recon2
    im = fas

grad = skimage.morphology.dilation(im, se) - skimage.morphology.erosion(im, se)
grad = np.int32(grad > 40) * grad
plt.imshow(grad, cmap="gray")
plt.show()

local_mini = skimage.feature.peak_local_max(255 - grad)
markers = np.zeros_like(grad, dtype=int)
for i, (row, col) in enumerate(local_mini, start=1):
    markers[row, col] = i
plt.imshow(local_mini, cmap="gray")
plt.show()

labels = skimage.segmentation.watershed(grad, markers, watershed_line=True)
plt.imshow(couleurs_alea(labels))
plt.show()

# visualization of the result
segm = labels.copy()
for i in range(segm.shape[0]):
    for j in range(segm.shape[1]):
        if segm[i, j] == 0:
            segm[i, j] = 255
        else:
            segm[i, j] = 0
            
# superimposition of segmentation contours on the original image
contourSup = np.maximum(segm, im)
plt.imshow(contourSup, cmap="gray")

In [ ]:
im = skimage.io.imread("Images/bulles.bmp")
plt.imshow(im, cmap="gray")
plt.show()

In [ ]:
im = skimage.io.imread("Images/laiton.bmp")
plt.imshow(im, cmap="gray")
plt.show()